In [1]:
%pip install sagemaker --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.30.2 \
    datasets[s3]==2.13.0 --quiet

  Using cached sagemaker-2.196.0.tar.gz (916 kB)
  Preparing metadata (setup.py) ... done
  Using cached pip-23.3.1-py3-none-any.whl (2.1 MB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached boto3-1.28.73-py3-none-any.whl (135 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached schema-0.7.5-py2.py3-none-any.whl (17 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.6/736.6 KB 5.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.3/83.3 KB 923.9 kB/s eta 0:00:0000:01
  Using cached tblib-1.7.0-py2.py3-none-any.whl (12 kB)
  Using cached s3transfer-0.7.0-py3-none-any.whl (79 kB)
  Using cached botocore-1.31.73-py3-none-any.whl (11.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.3 MB/s eta 0:00:000:00:01
  Using cached contextlib2-21.6.0-py2.py3-none-any.whl (13 kB)
  Created wheel for sagemaker: filename=sagemaker-2.196.0-py2.py3-none-any.whl size=1223221 sha256=ea05888a6effaf54ccc01d8c9109693baf4b8794383e

In [3]:
!pip install bs4

  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=7d761c4c2b74f32c3b4fcdda41471634169f967f34e51791f48e099e21f58c62
  Stored in directory: /root/.cache/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [4]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFace
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::984567242006:role/service-role/AmazonSageMaker-ExecutionRole-20231022T130295
sagemaker bucket: sagemaker-eu-north-1-984567242006
sagemaker session region: eu-north-1


# Preprocessing

In [12]:
from datasets import load_dataset
#from random import randrange

# Load dataset from the hub
import requests
from bs4 import BeautifulSoup
import pandas as pd

Add_on_dict = []
df = pd.DataFrame(columns=['Name','Link','Description'])
#title = soup.find('h2',{'class':'SearchResult-name'}).find('a').text

page = 1
while page != 6:
    url = f"https://addons.mozilla.org/en-US/firefox/search/?promoted=recommended&sort=random&type=extension&page={page}"
    print(url)
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, "html.parser")
    for title in soup.find_all("div",{'class':'SearchResult-contents'}):
        row_dict = {'Name' : title.find("a").text,'Link': title.find("a").get('href'), 'Description': title.find("p",{'class':"SearchResult-summary"}).get_text()}
        Add_on_dict.append(row_dict)
           
    page = page+1

https://addons.mozilla.org/en-US/firefox/search/?promoted=recommended&sort=random&type=extension&page=1
https://addons.mozilla.org/en-US/firefox/search/?promoted=recommended&sort=random&type=extension&page=2
https://addons.mozilla.org/en-US/firefox/search/?promoted=recommended&sort=random&type=extension&page=3
https://addons.mozilla.org/en-US/firefox/search/?promoted=recommended&sort=random&type=extension&page=4
https://addons.mozilla.org/en-US/firefox/search/?promoted=recommended&sort=random&type=extension&page=5


In [6]:
df = pd.DataFrame.from_dict(Add_on_dict)
#df.head()

import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())

### convert to Huggingface dataset
dataset = Dataset(pa.Table.from_pandas(df))

In [7]:
dataset

Dataset({
    features: ['Name', 'Link', 'Description'],
    num_rows: 108
})

In [8]:
def format_dolly(sample):
    instruction = f"### Description\n{sample['Description']}"
    response = f"### Answer\n{sample['Name']}\n{sample['Link']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [instruction, response] if i is not None])
    return prompt

In [9]:
from random import randrange

print(format_dolly(dataset[randrange(len(dataset))]))

### Description
Scroll page without touching scroll-bar! 
Press Middle (Right / Left) mouse button anywhere on the page to scroll just like with scrollbar.

Features also:
- "grab and drag" scrolling
- customizable scrollbars!
- the Momentum auto-scroll

### Answer
ScrollAnywhere
/en-US/firefox/addon/scroll_anywhere/?utm_source=addons.mozilla.org&utm_medium=referral&utm_content=featured


In [10]:
from transformers import AutoTokenizer

model_id = "tiiuae/falcon-40b" # sharded weights
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [13]:
from random import randint
from itertools import chain
from functools import partial



# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample


# apply prompt template per sample
dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
# print random sample
print(dataset[randint(0, len(dataset))]["text"])

# empty list to save remainder from batches to use in next batch
remainder = {"input_ids": [], "attention_mask": [], "token_type_ids": []}

def chunk(sample, chunk_length=2048):
    # define global remainder variable to save remainder from batches to use in next batch
    global remainder
    # Concatenate all texts and add remainder from previous batch
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    # get total number of tokens for batch
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # get max number of chunks for batch
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # Split by chunks of max_len.
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # add remainder to global variable for next batch
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # prepare labels
    result["labels"] = result["input_ids"].copy()
    return result


# tokenize and chunk dataset
lm_dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
).map(
    partial(chunk, chunk_length=2048),
    batched=True,
)

# Print total number of samples
print(f"Total number of samples: {len(lm_dataset)}")

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [13]:
# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/processed/dolly/train'
lm_dataset.save_to_disk(training_input_path)

print("uploaded data to:")
print(f"training dataset to: {training_input_path}")

uploaded data to:
training dataset to: s3://sagemaker-eu-north-1-984567242006/processed/dolly/train


In [15]:
import time
# define Training Job Name 
job_name = f'huggingface-qlora-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters ={
  'model_id': model_id,                                # pre-trained model
  'dataset_path': '/opt/ml/input/data/training', # path where sagemaker will save training dataset
  'epochs': 3,                                         # number of training epochs
  'per_device_train_batch_size': 4,                    # batch size for training
  'lr': 2e-4,                                          # learning rate used during training
}

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'run_clm.py',      # train script
    source_dir           = 'scripts',         # directory which includes all the files needed for training
    instance_type        = 'ml.m5.xlarge', # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size          = 300,               # the size of the EBS volume in GB
    transformers_version = '4.30',            # the transformers version used in the training job
    pytorch_version      = '1.0',            # the pytorch_version version used in the training job
    py_version           = 'py310',            # the python version used in the training job
    hyperparameters      =  hyperparameters,
    environment          = { "HUGGINGFACE_HUB_CACHE": "/tmp/.cache" }, # set env variable to cache models in /tmp
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [16]:
# define a data input dictonary with our uploaded s3 uris
data = {'training': training_input_path}

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=True)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


ValueError: Unsupported processor: cpu. You may need to upgrade your SDK version (pip install -U sagemaker) for newer processors. Supported processor(s): gpu.